# API wrappers

The OpenWeatherMap offers REST endpoints for querying current weather, forecasts, historical data, etc. However, accessing this data directly via the REST API requires handling multiple API calls, query parameters, and response parsing. The pyowm library abstracts these complexities and provides useful built-in functionalities.

After signing in to OpenWeatherMap retrieve your api key at https://home.openweathermap.org/api_keys

You will also need to install the pyowm package: pip install pyowm 

In [9]:
import requests
import pyowm
import json


api_key = '251cf6bdf97121546f2620ec0d8995bc'

## use case 1: managing API keys

In a raw rest API call you always have to manage credentials in each individual call. Wrappers usually store and manage the authentication for you

In [15]:
#You can get current weather data by making a GET request to an endpoint like:

params = {
    'appid' : api_key
}

response = requests.get('https://api.openweathermap.org/data/2.5/weather?q=London', params = params)

json.loads(response.text)

#but for every call you make using GET from now on you do need to add the parameters, since the raw API does not manage authentication for you

{'coord': {'lon': -0.1257, 'lat': 51.5085},
 'weather': [{'id': 801,
   'main': 'Clouds',
   'description': 'few clouds',
   'icon': '02d'}],
 'base': 'stations',
 'main': {'temp': 287.74,
  'feels_like': 286.87,
  'temp_min': 287.09,
  'temp_max': 290.1,
  'pressure': 1020,
  'humidity': 62,
  'sea_level': 1020,
  'grnd_level': 1015},
 'visibility': 10000,
 'wind': {'speed': 1.79, 'deg': 1, 'gust': 3.13},
 'clouds': {'all': 17},
 'dt': 1747571538,
 'sys': {'type': 2,
  'id': 2075535,
  'country': 'GB',
  'sunrise': 1747541060,
  'sunset': 1747597787},
 'timezone': 3600,
 'id': 2643743,
 'name': 'London',
 'cod': 200}

Most wrappers (pyowm included) include some way of initializing a session with the authentication key that you then don't need to type again.

Initialize pyowm with the default configuration. Thenopen the weather manager

Check out a snippet here: https://pyowm.readthedocs.io/en/latest/v3/code-recipes.html#weather_data

In [16]:
#your code here

from pyowm.owm import OWM

# Tu clave API de OpenWeatherMap
api_key = '251cf6bdf97121546f2620ec0d8995bc'

owm = OWM(api_key)
weather_mgr = owm.weather_manager()

## use case 2: Simplified calls

With the raw REST API, you'd have to build a URL manually, send the request, and parse the JSON response to get the current weather.

In [17]:
city = 'London'
url = f'http://api.openweathermap.org/data/2.5/weather?q={city}'

response = requests.get(url,params= params)
data = response.json()
temperature = data['main']['temp']
humidity = data['main']['humidity']
wind_speed = data['wind']['speed']

print(f"Temperature: {temperature}°C, Humidity: {humidity}%, Wind Speed: {wind_speed} m/s")

Temperature: 288.14°C, Humidity: 61%, Wind Speed: 2.24 m/s


Get the equivalent call as above for the city of London using the pyowm package

In [19]:
#your code here

from pyowm.owm import OWM

#API KEY
api_key = '251cf6bdf97121546f2620ec0d8995bc'
owm = OWM(api_key)

# open weather manager
mgr = owm.weather_manager()

# To check the weather of the city
observation = mgr.weather_at_place("London,GB")
weather = observation.weather

temperature = weather.temperature('celsius')['temp']  # temperatura en °C
humidity = weather.humidity                            # humedad en %
wind_speed = weather.wind()['speed']                   # velocidad del viento en m/s

print(f"Temperature: {temperature}°C, Humidity: {humidity}%, Wind Speed: {wind_speed} m/s")

Temperature: 15.08°C, Humidity: 60%, Wind Speed: 1.34 m/s


## use case 3: Combining and chaining calls

Wrappers often offer methods that make multiple calls to batch requests that make sense to batch. And often they offer methods that make sequences of calls that each returns information necessary to make the next call.

For example, to get a weather forecast for a specific city using the raw API you need to first geocode the city to get its latitude and longitude:

In [20]:
city = 'New York'
geocode_url = f'http://api.openweathermap.org/data/2.5/weather?q={city}'
geocode_response = requests.get(geocode_url,params=params).json()

lat = geocode_response['coord']['lat']
lon = geocode_response['coord']['lon']

Then, request the weather forecast for that latitude/longitude:

In [22]:
forecast_url = f'http://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}'
forecast_response = requests.get(forecast_url, params=params).json()

for entry in forecast_response['list']:
    print(f"Time: {entry['dt_txt']}, Temp: {entry['main']['temp']}°F")

Time: 2025-05-18 15:00:00, Temp: 291.87°F
Time: 2025-05-18 18:00:00, Temp: 292.95°F
Time: 2025-05-18 21:00:00, Temp: 293.49°F
Time: 2025-05-19 00:00:00, Temp: 289.78°F
Time: 2025-05-19 03:00:00, Temp: 288.54°F
Time: 2025-05-19 06:00:00, Temp: 286.44°F
Time: 2025-05-19 09:00:00, Temp: 286.38°F
Time: 2025-05-19 12:00:00, Temp: 287.64°F
Time: 2025-05-19 15:00:00, Temp: 291.41°F
Time: 2025-05-19 18:00:00, Temp: 294.66°F
Time: 2025-05-19 21:00:00, Temp: 294.42°F
Time: 2025-05-20 00:00:00, Temp: 290.48°F
Time: 2025-05-20 03:00:00, Temp: 287.62°F
Time: 2025-05-20 06:00:00, Temp: 285.71°F
Time: 2025-05-20 09:00:00, Temp: 284.86°F
Time: 2025-05-20 12:00:00, Temp: 285.59°F
Time: 2025-05-20 15:00:00, Temp: 289.35°F
Time: 2025-05-20 18:00:00, Temp: 292.94°F
Time: 2025-05-20 21:00:00, Temp: 294.22°F
Time: 2025-05-21 00:00:00, Temp: 291.07°F
Time: 2025-05-21 03:00:00, Temp: 289.63°F
Time: 2025-05-21 06:00:00, Temp: 288.53°F
Time: 2025-05-21 09:00:00, Temp: 287.03°F
Time: 2025-05-21 12:00:00, Temp: 2

Two calls: one for geocoding, one for forecasts.
But with pyowm, because this is a common operation, there is a method that handles the geocoding internally and then fetches the weather forecast in one step.

Get the above forecast in a single call using pyowm.

Hint: search for "forecast_at_place" in the code recipies of the documentation

In [26]:
#your code here

from pyowm.owm import OWM

api_key = '251cf6bdf97121546f2620ec0d8995bc'
owm = OWM(api_key)
mgr = owm.weather_manager()

forecast = mgr.forecast_at_place("New York,US", '3h')  # '3h' = to get infor every 3hr
weather_list = forecast.forecast.weathers

for weather in weather_list[:5]:  # Puedes limitar el número si quieres
    time = weather.reference_time('iso')
    temp = weather.temperature('celsius')['temp']
    print(f"Time: {time}, Temp: {temp}°C")

Time: 2025-05-18 15:00:00+00:00, Temp: 18.81°C
Time: 2025-05-18 18:00:00+00:00, Temp: 19.85°C
Time: 2025-05-18 21:00:00+00:00, Temp: 20.34°C
Time: 2025-05-19 00:00:00+00:00, Temp: 16.63°C
Time: 2025-05-19 03:00:00+00:00, Temp: 15.39°C


## use case 4: Convenience methods

Wrappers often offer built-in methods to handle common kinds of tasks related to the APIs, reducing the need for manual calculations.

for example converting units (e.g., temperature from Celsius to Fahrenheit) or working with more complex data requires manual conversion when using the raw API.

In [27]:
city = 'London'
url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}'

response = requests.get(url)
data = response.json()
temperature_celsius = data['main']['temp']
temperature_fahrenheit = (temperature_celsius * 9/5) + 32

print(f"Temperature in Celsius: {temperature_celsius}°C, Fahrenheit: {temperature_fahrenheit}°F")

Temperature in Celsius: 288.73°C, Fahrenheit: 551.714°F


But the pyowm wrapper offers built-in methods to handle these kinds of tasks, reducing the need for manual calculations.
Get the temperature both in Celcius and Farenheit using pyowm. Navigate the code recipes to figure out the inbuilt methods for this.

In [31]:
#your code here

from pyowm.owm import OWM

api_key = '251cf6bdf97121546f2620ec0d8995bc'
owm = OWM(api_key)
mgr = owm.weather_manager()

observation = mgr.weather_at_place("Madrid,ES")
weather = observation.weather

#temperatures in C and F
temp_c = weather.temperature('celsius')['temp']
temp_f = weather.temperature('fahrenheit')['temp']

print(f"Temperature in Celsius: {temp_c}°C")
print(f"Temperature in Fahrenheit: {temp_f}°F")


Temperature in Celsius: 25.62°C
Temperature in Fahrenheit: 78.12°F
